# Notebook Instructions:
This notebook uses dask to accelerate Array and DataFrame operations. Cell [4] starts up a dask client which connects to the scheduler located at the tcp address passed to its constructor. Client, scheduler and worker environments need to match in order 
for dask to work properly. Create a conda env from the config file "dask_environment.yaml", write a matching jupyter kernel from within that environment, then start a scheduler and workers from that same conda env. 

On the first run, you will need to authorize to tiled with your BNL domain account.

In [1]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
sid = 164307

In [3]:
import tiled
import databroker
print(tiled.__version__)
print(databroker.__version__)

0.1.0b17
2.0.0b46


In [4]:
import time as time
tstart = time.time()
import os
import pathlib
import sys
print(sys.version)
#import tpx3awkward as tpx
#import tpxspdc as ts
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl 
import pickle
import dask_histogram as dh
from fast_histogram import histogram1d, histogram2d
from scipy.optimize import curve_fit
import json
import h5py
import glob
import databroker

import pandas as pd
print(pd.__version__)
pd.set_option('display.max_columns', None)
pd.options.mode.copy_on_write = True 
import dask
print(dask.__version__)
import dask.dataframe as dd
from dask import persist
from distributed.diagnostics.plugin import UploadFile

from __future__ import print_function, absolute_import
from collections import OrderedDict, deque
from numba import njit, jit
import numba as nb
from numba import float64, int64, deferred_type, optional
from numba.core.runtime import rtsys
from numba.experimental import jitclass


3.12.7 | packaged by conda-forge | (main, Oct  4 2024, 16:05:46) [GCC 13.3.0]
2.2.3
2024.8.0


/nsls2/users/jgoodrich/conda_envs/dask_compatible/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# load data

In [5]:
from dask.distributed import Client

client = Client("tcp://10.65.4.102:8786")
client

<Client: 'tcp://10.65.4.102:8786' processes=8 threads=64, memory=2.25 TiB>

In [6]:
#inject distributed client code above
client.upload_file("tpxspdc.py")

{'tcp://10.65.4.102:34777': {'status': 'OK'},
 'tcp://10.65.4.102:34903': {'status': 'OK'},
 'tcp://10.65.4.102:39709': {'status': 'OK'},
 'tcp://10.65.4.102:39815': {'status': 'OK'},
 'tcp://10.65.4.102:41765': {'status': 'OK'},
 'tcp://10.65.4.102:44187': {'status': 'OK'},
 'tcp://10.65.4.102:44287': {'status': 'OK'},
 'tcp://10.65.4.102:46367': {'status': 'OK'}}

In [7]:
import tpx3awkward as tpx
import tpxspdc as ts

In [8]:
##### Modify these as needed!
# print(tiled.__version__)
# print(tiled.__file__)
### Basic scan info:
from tiled.client import from_uri
db = from_uri('https://tiled.nsls2.bnl.gov', 'dask')['chx']
db = db['raw']

r = db[sid] #check that we can access databroker through an SID, requires databroker moduler with version larger 2.x
r_md = r.metadata
r_md = r_md['start']
# print(r)
print(r_md)

scan_info = {
    'sid' : sid,
    'Epump': r_md['Epump'],
    'dd' : r_md['dd'], 
    'del_theta' : r_md['del_theta'],
    'theta' : r_md['theta'],
    'x_cen' : r_md['x_cen'],
    'y_cen' : r_md['y_cen']
}

files = tpx.converted_path(db[sid]['primary']['data']['tpx3_files_raw_filepaths'][0].compute(), cent=True)
for file in files:
    str(file).replace('Compressed_Data', 'assets').replace('_cent.h5', '.tpx3')
    if file.stat().st_size == 0:
        tpx.convert_tpx_file(Path(file), print_details=True)
        
print(files[0])


set_x = 254
set_y = 257


### Variables for controlling what cells run:
plot_basic_info = True
four_chip = False # if false, uses pixel by pixel method

### dp selection vars
dp_search = True
x_minus_offset = -1
x_plus_offset = 1
y_minus_offset = -1
y_plus_offset = 1

all_selections = True # should keep True for now until more condensed version is available (not fully implemented)


dirs = ['data/tot', 'data/pairs', 'data/selections']


base_path = "/nsls2/data/chx/proposals/2025-1/pass-316251/Results/"

# Iterate over the directories and create them if they don't exist
for dir in dirs:
   full_path = os.path.join(base_path, dir)
   pathlib.Path(full_path).mkdir(parents=True, exist_ok=True)

    
### Variables for saving intermediate dataframes:

first_run = True
save_tot = False
save_pairs = False
save_selections = False

# if not glob.glob(os.path.join(base_path, dirs[0], 'tot_*.h5')):
#     first_run = True
#     save_tot = True
#     save_pairs = True
#     save_selections = True
# else:
#     print("ToT selection data found, skipping ahead! Overwrite first_run manually to re-run ToT filtering")




### Selection variables
tot_offset = 25
dt_selection_range = 2 # in sigma_t
Etot_selection_range = 2 # in del_E t
dp_selection_range = 10 # in pixels

### Whether to find nominal t0 / E0 values or use the values specified below
find_nominals = True
t0 = 0
tsigma = 20
E0 = 15
Esigma = .4

### Variables for saving plots:
save_plots = False
plot_folder = 'figures' 
#pathlib.Path(os.path.join(base_path,plot_folder)).mkdir(parents=True, exist_ok=True)

### Variable for saving log results:
log_file = 'results.txt'

### We should analyzing data after it has been corrected by adding dummy pixels between chips.
npixels = 514 # By default, tpx3awkward.add_centroid_cols does this now.

pixel_to_mm = 0.055 # 55 microns is 0.055 millimeters
npixels_mm = npixels * pixel_to_mm

### Figure size parameters.
labelsize = 10
ticksize = 8
legendsize = 8
figsize = (14.16, 7.08)
figsize_sing = (3.54, 3.54)
dpi = 300
#mpl.rcParams['figure.dpi'] = dpi


{'uid': '8d98e823-eb84-4eec-b002-fd4e76d6995d', 'time': 1739449076.3871093, 'dd': 49.400000000000006, 'sample': 'Diamond (111)', 'Epump': 15, 'beamline_id': 'CHX', 'beam_position_dict': {'eiger4m': {'beam_center_x': 1808.0, 'beam_center_y': 1914.0, 'det_x': 172.65200000000002, 'det_y': -82.2485}, 'eiger500k': {'beam_center_x': 530.0, 'beam_center_y': 5.0, 'det_x': -31.2468, 'det_y': -145.0731}}, 'data_session': 'pass-316251', 'x_cen': 258, 'auto_pipeline': 'SPDC_pandas', 'start_datetime': '2025-02-10T12:46:54.372812', 'owner': 'xf11id', 'scan_id': 164307, 'nominal_phh': -10.41821183995863, 'current_phh': -10.4594513123, 'username': 'fluerasu', 'OAV_resolution [um_pixel]': 'N.A.', 'y_cen': 258, 'cycle': '2025-1', 'theta': 11.576, 'proposal': {'proposal_id': '316251', 'title': 'X-ray Quantum Coherent Scattering: A New Method for Beam-Sensitive Materials', 'type': 'Partner User', 'pi_name': 'Andrei Fluerasu'}, 'sample thickness [mm]': 0.5, 'delta_theta': -0.037282547141369804, 'user': 'qm

In [9]:
##### Load the data in, fix columns, mask out pixels if necessary, and channel them.

t1 = time.time()
print(f"Imports and setup done after {(t1-tstart):.1f} seconds.")

first_run = True
if first_run:
    print("Loading centroided data:")
    
    # cfpath = os.path.join(directory,fileuid+"_cent.h5")
    
    _df = dd.read_hdf(files, key='df', mode='r', sorted_index = False,chunksize=2000000, lock=False)#.repartition(npartitions=64)


    t2 = time.time()
    print(f"...loading complete after {(t2-t1):.1f} seconds. Fixing large pixel gap and additional centroid columns:")
    _df = ts.add_centroid_cols(_df)
    
    t2b = time.time()
    print(f"...gap and additional centroid columns complete after {(t2b-t2):.1f} seconds. Removing dead pixels:")
    
    # There are some noisy pixels behind the beamstop which we have to remove. Maybe should be done at an earlier step of the data processing.
    conditions = (
        ~((_df['xc'] >= 250) & (_df['xc'] <= 260) & (_df['yc'] >= 250) & (_df['yc'] <= 260))
        )

    _df = _df[conditions]

    t3 = time.time()
    print(f"...Dead pixels removed after {(t3-t2b):.1f} seconds. Channeling:")
    _df = ts.channel_optimized(_df)
    
    _df = client.persist(_df)

        
    print(f"DataFrame contains {len(_df.index)} total rows.")
    t4 = time.time()
    print(f"...channeling complete after {(t4-t3):.1f} seconds. Total elapsed time of this section is {(t4-t1):.1f} seconds.")

Imports and setup done after 7.1 seconds.
Loading centroided data:
...loading complete after 5.2 seconds. Fixing large pixel gap and additional centroid columns:
...gap and additional centroid columns complete after 0.0 seconds. Removing dead pixels:
...Dead pixels removed after 0.0 seconds. Channeling:


KilledWorker: Attempted to run task ('assign-e737d1fc663122597f91dd3bb738586f', 96) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://10.65.4.102:34827. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

In [ ]:
##### Plot some basic info of the raw data:

t4b = time.time()
if plot_basic_info and first_run:

   # nbins = 514
    
    hist_data, edges = dh.histogramdd(_df[['xc','yc']], range=((0, 513), (0, 513)), bins=(npixels,npixels))
    nbins_n = 10
    dmin_n = 0
    dmax_n = 10
    hist_data_n,edges_n = dh.histogram(_df['n'], bins = nbins_n, range = [dmin_n, dmax_n])

    dmin_t,dmax_t = dd.compute(_df['t_ns'].min(),_df['t_ns'].max())
    
    nbins_t = 10000
    hist_data_t,edges_t = dh.histogram(_df['t_ns'], bins = nbins_t, range = [dmin_t, dmax_t])
    
    nbins_ToT = 40
    dmin_ToT = 0
    dmax_ToT = 1000
    
    hist_data_1, edges_1 = dh.histogram(_df['ToT_max'], bins = nbins_ToT, range = [dmin_ToT, dmax_ToT])
    hist_data_2,edges_2 = dh.histogram(_df['ToT_sum'], bins=nbins_ToT, range = [dmin_ToT, dmax_ToT])
    
    hist_data, edges, hist_data_n,edges_n,hist_data_t,edges_t, hist_data_1, edges_1, hist_data_2, edges_2 = dd.compute(hist_data,edges, hist_data_n,edges_n,hist_data_t,edges_t, hist_data_1, edges_1, hist_data_2, edges_2)
    
    fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(9.5, 4))
    h = ax0.pcolormesh(hist_data)
    fig.colorbar(h, ax = ax0)
    h = ax1.pcolormesh(hist_data, norm=mpl.colors.LogNorm())
    fig.colorbar(h, ax = ax1)
    fig.tight_layout()
    plt.show()
    fig, axs = plt.subplots(2, 1, figsize=(9.5, 4))
    axs[0].stairs(hist_data.sum(axis=1), color='blue', fill=True, alpha=0.7)
    axs[0].set_title('Histogram of x')
    axs[0].set_xlabel('x')
    axs[0].set_ylabel('Frequency')
    
    axs[1].stairs(hist_data.sum(axis=0), color='green',fill=True, alpha=0.7)
    axs[1].set_title('Histogram of y')
    axs[1].set_xlabel('y')
    axs[1].set_ylabel('Frequency')
    plt.tight_layout()
    plt.show()
    
    
    
    
    fig, ax0 = plt.subplots(ncols=1, figsize=(5, 4))
    plt.stairs( hist_data_n, edges_n, linewidth = 1)
    plt.ylim([0, np.max(hist_data_n)*1.05])
    plt.xlim([dmin_n, dmax_n])
    plt.xlabel('events/cluster',fontsize = 12)
    plt.ylabel('counts',fontsize = 12)
    plt.show()
   
        
    fig, ax0 = plt.subplots(ncols=1, figsize=(10, 4))
    plt.stairs(hist_data_t, edges_t, color = 'k', edgecolor = 'k', fill=True)
    plt.xlabel('ToA [ns]',fontsize = 12)
    plt.ylabel('counts',fontsize = 12)
    plt.show()
    
    
    fig, ax0 = plt.subplots(ncols=1, figsize=(5, 4))
    plt.stairs(hist_data_1, edges_1,color= 'r', linewidth = 1, label='highest ToT/cluster')
    plt.stairs(hist_data_2, edges_2, color= 'b', linewidth = 1, label='cluster ToT sum')
    plt.ylim([0, np.max([hist_data_1, hist_data_2])*1.05])
    plt.xlabel('ToT [ns]',fontsize = 12)
    plt.ylabel('count',fontsize = 12)
    plt.legend()
    plt.show()
    
t5 = time.time()
print(f"Basic plots done after {(t5-t4b):.1f} seconds.")


# analysis

In [ ]:
##### ToT filtering

#centroided data sometimes contains points with y index larger than largest pixel - filter out here first
conditions = (
            (_df['y'] < 514)
        )

_df = _df[conditions]#.persist()


if first_run:
    t5 = time.time()
    
    print("Filtering...")
    if four_chip: # Basic method - apply a threshold to each chip.

        optimal_tots = [350, 375, 425, 425] # Set optimal ToTs (4 chip method).  tpxspdc also has pixel by pixel method
        optimal_tots = [375, 400, 450, 450] # +25 ns of original values
        df = ts.tot_filter_quad(_df, optimal_tots)

    else: # More advanced method - each pixel has its own threshold. Preferred method.

        with open("/nsls2/data/chx/legacy/analysis/2023_2/qmicroscope/Results/test", "rb") as fp:
        #with open("/nsls2/data/chx/legacy/analysis/2024_2/qmicroscope/jhanrahan/pixel_by_pixel/optimal_tot_values_12.7kv_2.npy", "rb") as fp:
            opt_tots = np.load(fp, allow_pickle=True)

            opt_tots = np.asarray(opt_tots, dtype=np.int32)# - tot_offset
            max_threshold = opt_tots.max()
            opt_tots = opt_tots - tot_offset
#            Reference data is 512x512, which doesn't consider the large gap between the four chips. Adjust here. Probably could just fix the opt_tots directly..
            expanded_tots = np.zeros((514, 514, 1))
            expanded_tots[:256, :256, :] = opt_tots[:256, :256, :]
            expanded_tots[:256, 258:, :] = opt_tots[:256, 256:, :]
            expanded_tots[258:, :256, :] = opt_tots[256:, :256, :]
            expanded_tots[258:, 258:, :] = opt_tots[256:, 256:, :]
            expanded_tots[256:258, :, :] = 600
            expanded_tots[:, 256:258, :] = 600

        df = ts.tot_filter_pixel(_df, expanded_tots, prints = True)
        
    t6 = time.time()
    print(df)
    print(f"...complete after {(t6-t5):.1f} seconds. Saving:")

    save_tot = True

    #df = df.persist()
    if save_tot:
        df.to_hdf(os.path.join(base_path,"data/tot/tot_{}.h5".format(scan_info['sid'])), key='df', mode='w', format='table', lock=False)
        with h5py.File(os.path.join(base_path,"data/tot/tot_{}.h5".format(scan_info['sid'])), 'a') as h5f:
            h5f.create_dataset('scan_info', data=np.array(json.dumps(scan_info), dtype='S'))
        print(f"...saved after {(time.time()-t6):.1f} seconds.")
    
else:
    print("Reading saved ToT-filtered data") 
    df = dd.read_hdf(os.path.join(base_path,"data/tot/tot_{}.h5".format(scan_info['sid'])), key='df', mode='r', lock=False)
    df = client.persist(df)

df_length = len(df)
print(f"ToT-filtered DataFrame contains {df_length} total rows.")

In [ ]:
#move this to tpxspdc.py soon!

linked_list_pair_search = False

if (linked_list_pair_search):
   

    node_type = deferred_type()

    spec = OrderedDict()
    spec['ind1'] = int64
    spec['ind2'] = int64
    spec['metric'] = float64
    spec['next'] = optional(node_type)

    @jitclass(spec)
    class LinkedNode(object):
        def __init__(self, ind1, ind2, metric, next):
            self.ind1 = ind1
            self.ind2 = ind2
            self.metric = metric
            self.next = next

        def prepend(self, ind1, ind2, metric):
            return LinkedNode(ind1, ind2, metric, self)

    node_type.define(LinkedNode.class_type.instance_type)
    potential_pairs = LinkedNode(0, 0, 0, None)

    # find potential pairs and calculate their metric
    # if metric_min is >0, then only add pairs who have a metric greater than metric min; otherwise, if it's 0 (or less than 0) than add all pairs
    @jit(nopython=True)
    def find_potential_pairs(x, t, tot, potential_pairs, time_tolerance=50, metric_min = 13.7):
        print("finding potential pairs")
        count = 0
        for i in range(0, t.size):       
            n_i = i+1
            while (n_i < t.size and (t[i] + time_tolerance) >= t[n_i]):
                #tot_sum = tot[i] + tot[n_i]
                #if tot_sum > 575-tot_sigma and tot_sum < 575+tot_sigma:
                if (x[i] <= 256 and x[n_i] > 256):
                    m = metric(t[i] - t[n_i], tot[i] + tot[n_i])
                    if metric_min > 0:
                        if m > metric_min:
                            potential_pairs = potential_pairs.prepend(i, n_i, m) 
                            count += 1
                elif (x[i] > 256 and x[n_i] < 256):
                    m = metric(t[i] - t[n_i], tot[i] + tot[n_i])
                    if metric_min > 0:
                        if m > metric_min:
                            potential_pairs = potential_pairs.prepend(n_i, i, m) 
                            count += 1
                    else:
                        potential_pairs = potential_pairs.prepend(n_i, i, m) 
                        count += 1

                n_i += 1

        return potential_pairs, count

    # move potential pairs from linked list to array for sorting
    @njit
    def arrange_pairs(potential_pairs, p_pairs, count):
        print("arranging pairs")
        for i in range(0, count):
            if potential_pairs.ind1 == potential_pairs.ind2:
                print(f"ind1 = ind2?! {potential_pairs.ind1}")
            p_pairs[i]['ind'] = potential_pairs.ind1
            p_pairs[i]['nind'] = potential_pairs.ind2
            p_pairs[i]['metric'] = potential_pairs.metric
            potential_pairs = potential_pairs.next

        return p_pairs

    # select pairs with best metric
    # would be nice to make this work with numba, was having trouble with set but think it's possible
    def select_pairs(p_pairs):
        print("selecting pairs")
        used_indices = set()
        paired_events = []
        for idx1, idx2, _ in p_pairs:
            if idx1 not in used_indices and idx2 not in used_indices:
                if idx1 == idx2:
                    print(f"idx1 = idx2?! {idx1}")
                paired_events.append((idx1, idx2))
                used_indices.add(idx1)
                used_indices.add(idx2)

        return paired_events

    # double gaussian function, parameters are subject to change
    @jit(nopython=True)
    def metric(x, y, A=1232.141, x0=0, y0=573.663, sig_x=21.221, sig_y=76.473, rho=0):
        sum_term = (x-x0)**2/(2*sig_x**2) + (y-y0)**2/(2*sig_y**2)
        correlation_term = 2*(x-x0)*(y-y_o)*rho/(sig_x*sig_y)
        return (A*np.exp(-1.0*(sum_term-correlation_term)/(2*(1-rho**2))))
    
    potential_pairs = LinkedNode(0, 0, 0, None)
    data = df.compute()
    x = data['xc'].to_numpy()
    y = data['yc'].to_numpy()
    t = data['t_ns'].to_numpy()
    tot = data['ToT_sum'].to_numpy()

    potential_pairs, count = find_potential_pairs(x, t, tot, potential_pairs)
    p_pairs = arrange_pairs(potential_pairs, np.zeros(count, dtype=[('ind', np.int64), ('nind', np.int64), ('metric', np.float32)]), count)
    p_pairs = np.sort(p_pairs, order='metric')
    p_pairs = np.flip(p_pairs) # do we need to do this? maybe better way to sort?
    paired_events = np.array(select_pairs(p_pairs)).astype(np.uint32)

    df_1 = data.iloc[paired_events[:,0]].reset_index(drop=True)
    df_2 = data.iloc[paired_events[:,1]].reset_index(drop=True)
    df_1.columns = [f'{col}_1' for col in df_1.columns]
    df_2.columns = [f'{col}_2' for col in df_2.columns]

    df_pairs = pd.concat([df_1, df_2], axis=1)
    df_pairs = ts.calculate_pair_info_pandas(df_pairs, x_cen = scan_info['x_cen'], y_cen = scan_info['y_cen'], Epump = scan_info['Epump'], dd = scan_info['dd'], del_theta = scan_info['del_theta'], theta = scan_info['theta'])
    df_pairs = ts.calculate_detuning_info(df_pairs, x_cen = scan_info['x_cen'], y_cen = scan_info['y_cen'], Epump = scan_info['Epump'], dd = scan_info['dd'], del_theta = scan_info['del_theta'], theta = scan_info['theta'])

    print(df_pairs)

    plt.hist(df_pairs['delta_t'], range=(-200,200), bins=400)

In [ ]:
##### Find pairs by smallest time difference and add more info.

t7 = time.time()
print("Finding pairs...")

#repartition if we detect a large number of partitions
if (df.npartitions > 100): 
    df = df.repartition(npartitions=len(client.scheduler_info()['workers']))#.persist()

df_pairs = ts.get_combine_pairs(df, scan_info, prints = True)
t8 = time.time()
print(f"Pairs found after {(t8-t7):.1f} seconds. Calculating detuning:")
df_pairs = ts.calculate_detuning_info(df_pairs, x_cen = scan_info['x_cen'], y_cen = scan_info['y_cen'], Epump = scan_info['Epump'], dd = scan_info['dd'], del_theta = scan_info['del_theta'], theta = scan_info['theta'])
t9 = time.time()
print(f"Detuning info found after {(t9-t8):.1f} seconds.")
print(df_pairs)
if save_pairs:
    print("Saving:")
    df_pairs.to_hdf(os.path.join(base_path,"data/pairs/pairs_{}.h5".format(scan_info['sid'])), key='df', mode='w')
    with h5py.File(os.path.join(base_path,"data/pairs/pairs_{}.h5".format(scan_info['sid'])), 'a') as h5f:
        h5f.create_dataset('scan_info', data=np.array(json.dumps(scan_info), dtype='S'))
t10 = time.time()
if save_pairs:
    print(f"...saved after {(t10-t9):.1f} seconds.")
print(f"Total elapsed time of this section is {(t10-t7):.1f} seconds.")

df_pairs_length = len(df_pairs)
print(f"Pairs found: {df_pairs_length}.")

In [ ]:
##### Analysis of dp selection value and center location. 
t11 = time.time()

update_pos = True
dp_search = True
if dp_search:
    coords = [[i, j] for i in range(set_x + x_minus_offset, set_x + x_plus_offset + 1) for j in range(set_y + y_minus_offset, set_y + y_plus_offset + 1)]
    dp_lists = ts.dp_search(df_pairs, scan_info, coords)
    #dp_lists = ts.dp_search_dask(client, df_pairs, scan_info, coords)
    
    best_metric_pos = np.nanargmax(dp_lists['best_metric'])
    best_metric = dp_lists['best_metric'][best_metric_pos]
    best_coord = dp_lists['coords'][best_metric_pos]
    best_dp = dp_lists['best_dp'][best_metric_pos]
    print("Best coordinate is most likely at {} with dp = {} pix and metric = {:.1f}".format(best_coord, best_dp, best_metric))
    print("Set coordinate was [{}, {}]".format(scan_info['x_cen'], scan_info['y_cen']))

    if update_pos:
        scan_info['x_cen'] = best_coord[0]
        scan_info['y_cen'] = best_coord[1]
        
t12 = time.time()
print(f"dp search concluded after {(t12-t11):.1f} seconds.")

In [ ]:
# ##### Perform in depth selections.

t13 = time.time()
loop_count = 0
dp_selection_range = 3.0
if all_selections: # do all combinations of the selections to try to find the best parameters.
    
    while loop_count < 2:
        
        # Loop 1 - initial selections. At the end, it updates the detuning angle based off E_total plot (which should match E_pump) and then runs again.
        
        print("On loop {}".format(loop_count))

        # delta_t selection
        print("--> df dt")
        df_dt, df_dt_stats = ts.dt_filter(df_pairs, dt_selection_range)

        print("\n--> df dt Etot")
        df_dt_Etot, df_dt_Etot_stats = ts.dE_filter(df_dt, Etot_selection_range)

        print("\n--> df dt Etot dp")
        df_dt_Etot_dp = ts.dp_filter(df_dt_Etot, dp_selection_range)

        print("\n--> df dt dp")
        df_dt_dp = ts.dp_filter(df_dt, dp_selection_range)

        print("\n--> df dt dp Etot")
        df_dt_dp_Etot, df_dt_dp_Etot_stats = ts.dE_filter(df_dt_dp, Etot_selection_range)

        # delta P selections
        print("--> df dp")
        df_dp = ts.dp_filter(df_pairs, dp_selection_range)

        print("--> df dp dt")
        df_dp_dt, df_dp_dt_stats = ts.dt_filter(df_dp, dt_selection_range)

        print("--> df dp dt Etot")
        df_dp_dt_Etot, df_dp_dt_Etot_stats = ts.dE_filter(df_dp_dt, Etot_selection_range)

        print("--> df dp Etot")
        print("HERE")
        df_dp_Etot, df_dp_Etot_stats = ts.dE_filter(df_dp, Etot_selection_range)

        print("--> df dp Etot dt")
        df_dp_Etot_dt, df_dp_Etot_dt_stats = ts.dt_filter(df_dp_Etot, dt_selection_range)

        # Etotal selections
        print("\n--> df Etot")
        df_Etot, df_Etot_stats = ts.dE_filter(df_pairs, Etot_selection_range)

        print("\n--> df Etot dt")
        df_Etot_dt, df_Etot_dt_stats = ts.dt_filter(df_Etot, dt_selection_range)

        print("\n--> df Etot dt dp")
        df_Etot_dt_dp = ts.dp_filter(df_Etot_dt, dp_selection_range)

        print("\n--> df Etot dp")
        df_Etot_dp = ts.dp_filter(df_Etot, dp_selection_range)

        print("\n--> df Etot dp dt")
        df_Etot_dp_dt, df_Etot_dp_dt_stats = ts.dt_filter(df_Etot_dp, dt_selection_range)

        if find_nominals: 

            # df_dp_dt_Etot -> E0 and sigma
            t0_bad = df_dp_dt_stats['x0']
            tsigma_bad = df_dp_dt_stats['sigma']
            print(t0_bad, tsigma_bad)

            E0 = df_dp_dt_Etot_stats['x0_1']
            Esigma = df_dp_dt_Etot_stats['sigma1']
            print(E0, Esigma)

            # dp_dp_Etot_dt -> t0 and sigma
            E0_bad = df_dp_Etot_stats['x0_1']
            Esigma_bad = df_dp_Etot_stats['sigma1']
            print(E0_bad, Esigma_bad)

            t0 = df_dp_Etot_dt_stats['x0']
            tsigma = df_dp_Etot_dt_stats['sigma']
            print(t0, tsigma)

        # first iteration - Slightly better values can be found by alternating whether applying time or energy selections first.
        df_dp_dt2 = df_dp[(df_dp['delta_t'] >= t0 - tsigma*dt_selection_range) & (df_dp['delta_t'] <= t0 + tsigma*dt_selection_range)]
        df_dp_dt2_Etot, df_dp_dt2_Etot_stats = ts.dE_filter(df_dp_dt2, Etot_selection_range)

        df_dp_Etot2 = df_dp[(df_dp['E_tot'] >= E0 - Esigma*Etot_selection_range) & (df_dp['E_tot'] <= E0 + Esigma*Etot_selection_range)]
        df_dp_Etot2_dt, df_dp_Etot2_dt_stats = ts.dt_filter(df_dp_Etot2, dt_selection_range)

        if save_selections and loop_count == 1:
            df_dp_Etot2_dt.to_hdf(os.path.join(base_path,"data/selections/selections_{}.h5".format(scan_info['sid'])), key='df', mode='w')
            with h5py.File(os.path.join(base_path,"data/selections/selections_{}.h5".format(scan_info['sid'])), 'a') as h5f:
                h5f.create_dataset('scan_info', data=np.array(json.dumps(scan_info), dtype='S'))

        df_Etot2 = df_pairs[(df_pairs['E_tot'] >= E0 - Esigma*Etot_selection_range) & (df_pairs['E_tot'] <= E0 + Esigma*Etot_selection_range)]
        df_Etot2_dt, df_Etot2_dt_stats = ts.dt_filter(df_Etot2, dt_selection_range)

       # print(len(df_dp_dt2_Etot))
        print(len(df_dp_Etot2_dt))
        
        if loop_count == 0:
            
            # If at the end of the first loop, find the optimal delta theta value to run again.    
            optimal_del_theta = ts.get_optimal_del_theta(df_dp_Etot2_dt, scan_info)[0]
            print(optimal_del_theta)
            old_del_theta = scan_info['del_theta'] 
            scan_info['del_theta'] = optimal_del_theta # Update
            
            t71 = time.time()
            print("Finding new pairs with updated scan_info.")
            df_pairs = ts.get_pairs(df, chans1 = [1,4], chans2 = [2,3]).compute()
            t81 = time.time()
            print(f"Pairs found after {(t81-t71):.1f} seconds. Calculating pair info...")
            df_pairs = ts.calculate_pair_info_pandas(df_pairs, x_cen = scan_info['x_cen'], y_cen = scan_info['y_cen'], Epump = scan_info['Epump'], dd = scan_info['dd'], del_theta = scan_info['del_theta'], theta = scan_info['theta'])
            t91 = time.time()
            print(f"Basic pair info found after {(t91-t81):.1f} seconds. Calculating detuning info...")
            df_pairs = ts.calculate_detuning_info(df_pairs, x_cen = scan_info['x_cen'], y_cen = scan_info['y_cen'], Epump = scan_info['Epump'], dd = scan_info['dd'], del_theta = scan_info['del_theta'], theta = scan_info['theta'])
            
            if save_pairs:
                print("Saving:")
                t10 = time.time()
                df_pairs.to_hdf(os.path.join(base_path,"data/pairs/pairs_{}.h5".format(scan_info['sid'])), key='df', mode='w')
                with h5py.File(os.path.join(base_path,"data/pairs/pairs_{}.h5".format(scan_info['sid'])), 'a') as h5f:
                    h5f.create_dataset('scan_info', data=np.array(json.dumps(scan_info), dtype='S'))
                print(f"...saved after {(time.time()-t10):.1f} seconds.")
                
        loop_count = loop_count + 1
        
print("Initial detuning angle: {}".format(old_del_theta))
print("Optimal detuning angle: {}".format(optimal_del_theta))
t14 = time.time()
print(f"Selections concluded after {(t14-t13):.1f} seconds.")

# plots below 
## these are just examples/stuff we made for the figure
## many other sorts of plots could be generated from the dataframes!
## for example, distance of pairs histogram could be generated using df['dist']
## if there are other things we want to calculate, we can add the functions to tpxspdc.calculate_pair_info()!
### -------------------

## raw data

In [ ]:
fname = 'efig1a_'+plot_folder
binsize = 1

if first_run:
    if not plot_basic_info:
        hist_data, edges = dh.histogramdd(_df[['yc','xc']], range=((0, npixels), (0, npixels)), bins=(int(npixels/binsize),int(npixels/binsize)))
        hist_data,edges = dd.compute(hist_data,edges)
    #this has already been computed in cell 6

    xedges = edges[0]
    yedges = edges[1]
    xedges_mm = xedges * pixel_to_mm
    yedges_mm = yedges * pixel_to_mm

    plt.figure(figsize=figsize_sing)

    plt.pcolormesh(xedges_mm, yedges_mm, hist_data)

    plt.xlabel('x-position (mm)', fontsize=labelsize)
    plt.ylabel('y-position (mm)', fontsize=labelsize)
    plt.tick_params(axis='both', which='major', labelsize=ticksize)
    plt.gca().set_aspect('equal')

    cbar = plt.colorbar(shrink=0.72)
    cbar.ax.tick_params(labelsize=ticksize)
    plt.tight_layout()
    if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
    plt.show()
    plt.close()

## delta_t selections

In [ ]:
fname = 'efig1b_'+plot_folder
binsize = 8

hist, xedges, yedges = ts.histogram2d_fast(np.concatenate((df_dt['xc_1'], df_dt['xc_2'])), np.concatenate((df_dt['yc_1'], df_dt['yc_2'])), bins=int(npixels/binsize), range=[(0, npixels), (0, npixels)])
xedges_mm = xedges * pixel_to_mm
yedges_mm = yedges * pixel_to_mm

plt.figure(figsize=figsize_sing)

plt.pcolormesh(xedges_mm, yedges_mm, hist)

plt.xlabel('x-position (mm)', fontsize=labelsize)
plt.ylabel('y-position (mm)', fontsize=labelsize)
plt.tick_params(axis='both', which='major', labelsize=ticksize)
plt.gca().set_aspect('equal')

cbar = plt.colorbar(shrink=0.72)
cbar.ax.tick_params(labelsize=ticksize)
plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

## delta_t and E_tot ring

In [ ]:
fname = 'efig1c_'+plot_folder
binsize = 8

# Maybe this could go into tpxsdc. Should use fast_histogram
hist, xedges, yedges = ts.histogram2d_fast(np.concatenate((df_Etot2_dt['xc_1'], df_Etot2_dt['xc_2'])), np.concatenate((df_Etot2_dt['yc_1'], df_Etot2_dt['yc_2'])), bins=int(npixels/binsize), range=[(0, npixels), (0, npixels)])
xedges_mm = xedges * pixel_to_mm
yedges_mm = yedges * pixel_to_mm

plt.figure(figsize=figsize_sing)

plt.pcolormesh(xedges_mm, yedges_mm, hist)

plt.xlabel('x-position (mm)', fontsize=labelsize)
plt.ylabel('y-position (mm)', fontsize=labelsize)
plt.tick_params(axis='both', which='major', labelsize=ticksize)
plt.gca().set_aspect('equal')

cbar = plt.colorbar(shrink = 0.72)
cbar.ax.tick_params(labelsize=ticksize)
plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

## delta_t and Etot and dp ring (final ring)

In [ ]:
fname = 'fig1a_'+plot_folder
binsize = 8

# Maybe this could go into tpxsdc. Should use fast_histogram
hist, xedges, yedges = ts.histogram2d_fast(np.concatenate((df_dp_Etot2_dt['xc_1'], df_dp_Etot2_dt['xc_2'])), np.concatenate((df_dp_Etot2_dt['yc_1'], df_dp_Etot2_dt['yc_2'])), bins=int(npixels/binsize), range=[(0, npixels), (0, npixels)])
xedges_mm = xedges * pixel_to_mm
yedges_mm = yedges * pixel_to_mm

plt.figure(figsize=figsize_sing)

plt.pcolormesh(xedges_mm, yedges_mm, hist)

plt.xlabel('x-position (mm)', fontsize=labelsize)
plt.ylabel('y-position (mm)', fontsize=labelsize)
plt.tick_params(axis='both', which='major', labelsize=ticksize)
plt.gca().set_aspect('equal')

cbar = plt.colorbar(shrink = 0.72)
cbar.ax.tick_params(labelsize=ticksize)
plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
fname = 'fig1b_'+plot_folder

hist, xedges, yedges = ts.histogram2d_fast(np.concatenate((df_dp_Etot2_dt['xc_1_p'], df_dp_Etot2_dt['xc_2'])), np.concatenate((df_dp_Etot2_dt['yc_1_p'], df_dp_Etot2_dt['yc_2'])), bins=int(npixels/binsize), range=[(0, npixels), (0, npixels)])
xedges_mm = xedges * pixel_to_mm
yedges_mm = yedges * pixel_to_mm

plt.figure(figsize=figsize_sing)

plt.pcolormesh(xedges_mm, yedges_mm, hist)

plt.xlabel('x-position (mm)', fontsize=labelsize)
plt.ylabel('y-position (mm)', fontsize=labelsize)
plt.tick_params(axis='both', which='major', labelsize=ticksize)
plt.gca().set_aspect('equal')

cbar = plt.colorbar(shrink = 0.72)
cbar.ax.tick_params(labelsize=ticksize)
plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()

## delta t histograms

In [ ]:
fname = 'test2_'+plot_folder

set_color = 'red'; width = 3.0; alpha = 1.0; fmt = "-"

from matplotlib.cm import get_cmap
plt.figure(figsize=figsize_sing)

plt.plot(df_dt_stats['h_centers'], df_dt_stats['h_counts'], '.', color='blue', markersize=3.0)
plt.plot(df_dt_stats['model_x_vals'], df_dt_stats['model_y_vals'], '--', color='orange', linewidth=2.0, label='ToT selections only:\nN = {:.0f}, SN = {:.2f}'.format(df_dt_stats['N'], df_dt_stats['SN']))
"""
            label="ToT selections only: \n $N_{{pairs}}$ = {:.0f} \u00B1 {:.0f} \n $t_0$ = {:.2f} \u00B1 {:.2f} ns \n $\sigma$ = {:.2f} \u00B1 {:.2f} ns \n SNR = {:.2f} \u00B1 {:.2f}".format(
                abs(dt_N), abs(dt_N_err), dt_t0, dt_t0_err, abs(dt_sigma), dt_sigma_err, dt_SN, dt_SN_err))
"""
plt.plot(df_Etot2_dt_stats['h_centers'], df_Etot2_dt_stats['h_counts'], 's', color='black', markersize=2.0)
plt.plot(df_Etot2_dt_stats['model_x_vals'], df_Etot2_dt_stats['model_y_vals'], '--', color='red', linewidth=2.0, label='ToT & $E_{{total}}$ selections:\nN = {:.0f}, SN = {:.2f}'.format(df_Etot2_dt_stats['N'], df_Etot2_dt_stats['SN']))
"""
            label="ToT & $E_{{total}}$ = 15 keV selections: \n $N_{{pairs}}$ = {:.0f} \u00B1 {:.0f} \n $t_0$ = {:.2f} \u00B1 {:.2f} ns \n $\sigma$ = {:.2f} \u00B1 {:.2f} ns \n SNR = {:.2f} \u00B1 {:.2f}".format(
                abs(dtE_N), abs(dtE_N_err), abs(dtE_t0), dtE_t0_err, abs(dtE_sigma), dtE_sigma_err, dtE_SN, dtE_SN_err))
"""
                
plt.plot(df_dp_Etot2_dt_stats['h_centers'], df_dp_Etot2_dt_stats['h_counts'], 's', color='green', markersize=3.0)
plt.plot(df_dp_Etot2_dt_stats['model_x_vals'], df_dp_Etot2_dt_stats['model_y_vals'], '--', color='purple', linewidth=2.0, label='ToT & $E_{{total}}$ & dp < {} selections:\nN = {:.0f}, SN = {:.2f}'.format(dp_selection_range,df_dp_Etot2_dt_stats['N'],df_dp_Etot2_dt_stats['SN']))

plt.xlabel("$\Delta t$ (ns)", fontsize=labelsize)
plt.ylabel("Counts", fontsize=labelsize)
plt.tick_params(axis='both', which='major', labelsize=ticksize)
plt.xlim([-250, 250])

#plt.legend(fontsize=8)

plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

print("Time: N = {:.0f}, SN = {:.2f}".format(df_dt_stats['N'], df_dt_stats['SN']))
print("Time & Energy: N = {:.0f}, SN = {:.2f}".format(df_Etot2_dt_stats['N'], df_Etot2_dt_stats['SN']))
print("Time, Energy, Momentum: N = {:.0f}, SN = {:.1f}".format(df_dp_Etot2_dt_stats['N'], df_dp_Etot2_dt_stats['SN']))

## dr positions / closest position on line segment

In [ ]:
#hist_data_t,edges_t = plt.plot(df_dp_Etot2_dt['t_ns_1'],range = [dmin_t, dmax_t])573.663
# def metric(x, y, A=1232.141, x0=0, y0=0, sig_x=21.221, sig_y=76.473):
#         sum_term = (x-x0)**2/(2*sig_x**2) + (y-y0)**2/(2*sig_y**2)
#         return (A*np.exp(-1.0*sum_term))
# def metric(xy, A=1, x0=0, y0=0, sig_x=21.221, sig_y=76.473, v12=-.3):
#     x, y = xy
#     x0 = float(x0)
#     y0 = float(y0)
#     rho = v12/(sig_x*sig_y)
#     sum_term = (x-x0)**2/(sig_x**2) + (y-y0)**2/(sig_y**2)
#     correlation_term = 2*(x-x0)*(y-y0)*rho/(sig_x*sig_y)
#     normalization = 2*np.pi*sig_x*sig_y*np.sqrt(1-(rho*rho))
#     val = (A/normalization)*np.exp(-1.0*(sum_term-correlation_term)/(2*(1-rho**2)))
#     return val.ravel()

# def metric_noravel(xy, A=1,x0=0, y0=0, sig_x=21.221, sig_y=76.473, v12=-.3):
#     x, y = xy
#     x0 = float(x0)
#     y0 = float(y0)
#     rho = v12/(sig_x*sig_y)
#     sum_term = (x-x0)**2/(sig_x**2) + (y-y0)**2/(sig_y**2)
#     correlation_term = 2*(x-x0)*(y-y0)*rho/(sig_x*sig_y)
#     normalization = 2*np.pi*sig_x*sig_y*np.sqrt(1-(rho*rho))
#     val = (A/normalization)*np.exp(-1.0*(sum_term-correlation_term)/(2*(1-rho**2)))
#     return val

# #nx, ny = (80, 800)
# x = np.linspace(-40, 40, 81)
# y = np.linspace(-400, 400, 801)
# xv, yv = np.meshgrid(x, y)
# print(yv)

# bound = ([1,-1, -1, 10, 100, -1600],[5,10, 10, 200, 200, 0.45])
# print(len(df_dp_Etot2_dt.index))
# hist_data = plt.hist(df_dp_Etot2_dt['ToT_sum_1'], range=(df_dp_Etot2_dt['ToT_sum_1'].min(),df_dp_Etot2_dt['ToT_sum_1'].max()), bins=100)
# hist2_data = plt.hist(df_dp_Etot2_dt['ToT_sum_2'], range=(df_dp_Etot2_dt['ToT_sum_2'].min(),df_dp_Etot2_dt['ToT_sum_2'].max()), bins=100)
# plt.show()
# hist_data = plt.hist(df_dp_Etot2_dt['ToT_max_1'], range=(df_dp_Etot2_dt['ToT_max_1'].min(),df_dp_Etot2_dt['ToT_max_1'].max()), bins=100)
# hist2_data = plt.hist(df_dp_Etot2_dt['ToT_max_2'], range=(df_dp_Etot2_dt['ToT_max_2'].min(),df_dp_Etot2_dt['ToT_max_2'].max()), bins=100)
# plt.show()
# hist_data3 = plt.hist2d(df_dp_Etot_dt['t_1']-df_dp_Etot_dt['t_2'], df_dp_Etot_dt['ToT_sum_1'] - df_dp_Etot_dt['ToT_sum_2'], range=((-40,40),(-400,400)), bins=(40,32) )
# #plt.plot(xv, yv, marker='o', color='k', linestyle='none')

# p_guess = (1, 0.1, 0, 21, 100, -400)
# hx = np.linspace(hist_data3[2][0],hist_data3[2][-2],len(hist_data3[2])-1)
# hy = np.linspace(hist_data3[1][0],hist_data3[1][-2],len(hist_data3[1])-1)

# grid = np.array(np.meshgrid(hx,hy))
# print(grid)
# print(hist_data3[0])
# popt, pcov = curve_fit(metric, grid, hist_data3[0].ravel()/hist_data3[0].sum(), p0 = p_guess, maxfev = 5000, bounds=bound)
# A, xp, yp, sx, sy, r = popt
# print(popt)
# #print(xp, yp, sx, sy, r)
# plt.contour(xv, yv, metric_noravel((xv,yv),A,xp,yp,sx,sy,r))
# plt.show()

# print(df_dp_Etot2_dt['ToT_sum_1'])

In [ ]:
fname = 'fig3a_'+plot_folder

# Should use fast_histogram

cen_min = 220
cen_max = 300
binsize = 2

fig, ax = plt.subplots(ncols = 2, nrows = 2, figsize = figsize_sing, gridspec_kw = {"width_ratios": [3,0.5], "height_ratios": [1,5]})

ax0 = ax[1,0]
ax1 = ax[1,1]
ax2 = ax[0,0]
ax[0,1].axis("off")

hist, xedge, yedge = ts.histogram2d_fast(df_Etot2_dt['x_cen'], df_Etot2_dt['y_cen'], bins = int((cen_max-cen_min)/binsize), range=[[cen_min, cen_max], [cen_min, cen_max]])
h = ax0.imshow(hist, interpolation='none', origin='lower',extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]],aspect='auto')

plt.subplots_adjust(wspace = 0.2)
plt.subplots_adjust(hspace = 0.15)
ax1.hist(df_Etot2_dt['y_cen'], bins = np.arange(cen_min, cen_max + binsize, binsize), color='black', range = (cen_min, cen_max), histtype = 'step', orientation = "horizontal")
ax2.hist(df_Etot2_dt['x_cen'], bins = np.arange(cen_min, cen_max + binsize, binsize), color='black', range = (cen_min, cen_max), histtype = 'step')
ax0.tick_params(axis='both', which='major', labelsize=ticksize)
ax1.tick_params(axis='both', which='major', labelsize=ticksize)#ax1.yaxis.set_visible(False)
ax1.yaxis.set_visible(False)
ax1.set_ylim([cen_min, cen_max])
ax1.set_xlim([0, 300])
ax1.set_xlabel("count")
ax2.set_xlim([cen_min, cen_max])
ax2.set_ylim([0, 350])
ax2.set_ylabel("count")
ax2.xaxis.set_visible(False)
ax2.tick_params(axis='both', which='major', labelsize=ticksize)
ax0.set_xlabel("center x-position (pixel)", fontsize = labelsize)
ax0.set_ylabel("center y-position (pixel)", fontsize = labelsize)
plt.tight_layout() 
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
fname = 'fig3a_'+plot_folder

# Should use fast_histogram

cen_min = 220
cen_max = 300
binsize = 2

fig, ax = plt.subplots(ncols = 2, nrows = 2, figsize = figsize_sing, gridspec_kw = {"width_ratios": [3,0.5], "height_ratios": [1,5]})

ax0 = ax[1,0]
ax1 = ax[1,1]
ax2 = ax[0,0]
ax[0,1].axis("off")

hist, xedge, yedge = ts.histogram2d_fast(df_dp_Etot2_dt['x_cen'], df_dp_Etot2_dt['y_cen'], bins = int((cen_max-cen_min)/binsize), range=[[cen_min, cen_max], [cen_min, cen_max]])
h = ax0.imshow(hist, interpolation='none', origin='lower',extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]],aspect='auto')

plt.subplots_adjust(wspace = 0.2)
plt.subplots_adjust(hspace = 0.15)
ax1.hist(df_dp_Etot2_dt['y_cen'], bins = np.arange(cen_min, cen_max + binsize, binsize), color='black', range = (cen_min, cen_max), histtype = 'step', orientation = "horizontal")
ax2.hist(df_dp_Etot2_dt['x_cen'], bins = np.arange(cen_min, cen_max + binsize, binsize), color='black', range = (cen_min, cen_max), histtype = 'step')
ax0.tick_params(axis='both', which='major', labelsize=ticksize)
ax1.tick_params(axis='both', which='major', labelsize=ticksize)#ax1.yaxis.set_visible(False)
ax1.yaxis.set_visible(False)
ax1.set_ylim([cen_min, cen_max])
ax1.set_xlim([0, 300])
ax1.set_xlabel("count")
ax2.set_xlim([cen_min, cen_max])
ax2.set_ylim([0, 350])
ax2.set_ylabel("count")
ax2.xaxis.set_visible(False)
ax2.tick_params(axis='both', which='major', labelsize=ticksize)
ax0.set_xlabel("center x-position (pixel)", fontsize = labelsize)
ax0.set_ylabel("center y-position (pixel)", fontsize = labelsize)
plt.tight_layout() 
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
fname = 'fig3b_'+plot_folder

cen_min = 220
cen_max = 300
binsize = 2

fig, ax = plt.subplots(ncols = 2, nrows = 2, figsize = figsize_sing, gridspec_kw = {"width_ratios": [3,0.5], "height_ratios": [1,5]})

ax0 = ax[1,0]
ax1 = ax[1,1]
ax2 = ax[0,0]
ax[0,1].axis("off")

hist, xedge, yedge = ts.histogram2d_fast(df_Etot2_dt['xp'], df_Etot2_dt['yp'], bins = int((cen_max-cen_min)/binsize), range=[[cen_min, cen_max], [cen_min, cen_max]])
h = ax0.imshow(hist, interpolation='none', origin='lower',extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]],aspect='auto')

# h = ax0.hist2d(df_dt_Etot['xp'], df_dt_Etot['yp'], bins = ((np.arange(cen_min, cen_max + binsize, binsize)), (np.arange(cen_min, cen_max + binsize, binsize))))
plt.subplots_adjust(wspace = 0.2)
plt.subplots_adjust(hspace = 0.15)
ax1.hist(df_Etot2_dt['yp'], bins = np.arange(cen_min, cen_max + binsize, binsize), color='black', range = (cen_min, cen_max), histtype = 'step', orientation = "horizontal")
ax2.hist(df_Etot2_dt['xp'], bins = np.arange(cen_min, cen_max + binsize, binsize), color='black', range = (cen_min, cen_max), histtype = 'step')
ax0.tick_params(axis='both', which='major', labelsize=ticksize)
ax1.tick_params(axis='both', which='major', labelsize=ticksize)#ax1.yaxis.set_visible(False)
ax1.yaxis.set_visible(False)
ax1.set_ylim([cen_min, cen_max])
ax1.set_xlim([0, 3000])
ax1.set_xlabel("count")
ax2.set_xlim([cen_min, cen_max])
ax2.set_ylim([0, 3500])
ax2.set_ylabel("count")
ax2.xaxis.set_visible(False)
ax2.tick_params(axis='both', which='major', labelsize=ticksize)
ax0.set_xlabel("closest x-position (pixel)", fontsize = labelsize)
ax0.set_ylabel("closest y-position (pixel)", fontsize = labelsize)
plt.tight_layout() 
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
fname = 'fig3b_'+plot_folder

cen_min = 220
cen_max = 300
binsize = 2

fig, ax = plt.subplots(ncols = 2, nrows = 2, figsize = figsize_sing, gridspec_kw = {"width_ratios": [3,0.5], "height_ratios": [1,5]})

ax0 = ax[1,0]
ax1 = ax[1,1]
ax2 = ax[0,0]
ax[0,1].axis("off")

hist, xedge, yedge = ts.histogram2d_fast(df_dp_dt2_Etot['xp'], df_dp_dt2_Etot['yp'], bins = int((cen_max-cen_min)/binsize), range=[[cen_min, cen_max], [cen_min, cen_max]])
h = ax0.imshow(hist, interpolation='none', origin='lower',extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]],aspect='auto')

# h = ax0.hist2d(df_dt_Etot['xp'], df_dt_Etot['yp'], bins = ((np.arange(cen_min, cen_max + binsize, binsize)), (np.arange(cen_min, cen_max + binsize, binsize))))
plt.subplots_adjust(wspace = 0.2)
plt.subplots_adjust(hspace = 0.15)
ax1.hist(df_dp_dt2_Etot['yp'], bins = np.arange(cen_min, cen_max + binsize, binsize), color='black', range = (cen_min, cen_max), histtype = 'step', orientation = "horizontal")
ax2.hist(df_dp_dt2_Etot['xp'], bins = np.arange(cen_min, cen_max + binsize, binsize), color='black', range = (cen_min, cen_max), histtype = 'step')
ax0.tick_params(axis='both', which='major', labelsize=ticksize)
ax1.tick_params(axis='both', which='major', labelsize=ticksize)#ax1.yaxis.set_visible(False)
ax1.yaxis.set_visible(False)
ax1.set_ylim([cen_min, cen_max])
ax1.set_xlim([0, 3000])
ax1.set_xlabel("count")
ax2.set_xlim([cen_min, cen_max])
ax2.set_ylim([0, 3500])
ax2.set_ylabel("count")
ax2.xaxis.set_visible(False)
ax2.tick_params(axis='both', which='major', labelsize=ticksize)
ax0.set_xlabel("closest x-position (pixel)", fontsize = labelsize)
ax0.set_ylabel("closest y-position (pixel)", fontsize = labelsize)
plt.tight_layout() 
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

## energy and angle anti-correlations

In [ ]:
fname = 'fig4_'+plot_folder

fig, axs = plt.subplots(1, 2, figsize=figsize, gridspec_kw={'width_ratios': [1.6, 1]})

hist, xedge, yedge = ts.histogram2d_fast(df_dt['E_1'], df_dt['E_2'], range=((4, 11), (4, 11)), bins=100)
im = axs[0].imshow(hist, interpolation='none', origin='lower', extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]], aspect='auto')

# im = axs[0].hist2d(df_dt['E_1'], df_dt['E_2'], range=((4, 11), (4, 11)), bins=70)
axs[0].set_xlabel('E$_{1}$ (keV)', fontsize=labelsize)
axs[0].set_ylabel('E$_{2}$ (keV)', fontsize=labelsize)
axs[0].tick_params(axis='both', which='major', labelsize=ticksize)
axs[0].set_aspect('equal')  # Set aspect ratio to be equal for a square plot
cbar = fig.colorbar(im, ax=axs[0], shrink=.96)
cbar.ax.tick_params(labelsize=ticksize)
axs[0].annotate('a)', xy=(0, 1), xycoords='axes fraction', xytext=(-40, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)

axs[1].plot(df_dt_Etot_stats['h_centers'], df_dt_Etot_stats['h_counts'], '.', color='black')
axs[1].plot(df_dt_Etot_stats['model_x_vals'], df_dt_Etot_stats['model_y_vals'], '--', color='blue',
            label="$N_{{pairs}}$ = {:.0f} \u00B1 {:.0f} \n $E_0$ = {:.1f} keV \n  $\sigma$ = {:.2f}  keV".format(
                abs(df_dt_Etot_stats['N']), abs(df_dt_Etot_stats['N_err']), df_dt_Etot_stats['x0_2'], abs(df_dt_Etot_stats['sigma2'])))
axs[1].set_xlabel("E$_1$ + E$_2$ (keV)", fontsize=labelsize)
axs[1].set_ylabel("counts", fontsize=labelsize)
axs[1].tick_params(axis='both', which='major', labelsize=ticksize)
axs[1].set_xlim((10, 20))
axs[1].annotate('b)', xy=(0, 1), xycoords='axes fraction', xytext=(-50, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)

plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
fname = 'fig4_'+plot_folder

fig, axs = plt.subplots(1, 2, figsize=figsize, gridspec_kw={'width_ratios': [1.6, 1]})

hist, xedge, yedge = ts.histogram2d_fast(df_dp_dt2['E_1'], df_dp_dt2['E_2'], range=((4, 11), (4, 11)), bins=70)
im = axs[0].imshow(hist, interpolation='none', origin='lower', extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]], aspect='auto')

# im = axs[0].hist2d(df_dt['E_1'], df_dt['E_2'], range=((4, 11), (4, 11)), bins=70)
axs[0].set_xlabel('E$_{1}$ (keV)', fontsize=labelsize)
axs[0].set_ylabel('E$_{2}$ (keV)', fontsize=labelsize)
axs[0].tick_params(axis='both', which='major', labelsize=ticksize)
axs[0].set_aspect('equal')  # Set aspect ratio to be equal for a square plot
cbar = fig.colorbar(im, ax=axs[0], shrink=.96)
cbar.ax.tick_params(labelsize=ticksize)
axs[0].annotate('a)', xy=(0, 1), xycoords='axes fraction', xytext=(-40, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)

axs[1].plot(df_dp_dt2_Etot_stats['h_centers'], df_dp_dt2_Etot_stats['h_counts'], '.', color='black')
axs[1].plot(df_dp_dt2_Etot_stats['model_x_vals'], df_dp_dt2_Etot_stats['model_y_vals'], '--', color='blue',
            label="$N_{{pairs}}$ = {:.0f} \u00B1 {:.0f} \n $E_0$ = {:.1f} keV \n  $\sigma$ = {:.2f}  keV".format(
                abs(df_dt_Etot_stats['N']), abs(df_dt_Etot_stats['N_err']), df_dt_Etot_stats['x0_2'], abs(df_dt_Etot_stats['sigma2'])))
axs[1].set_xlabel("E$_1$ + E$_2$ (keV)", fontsize=labelsize)
axs[1].set_ylabel("counts", fontsize=labelsize)
axs[1].tick_params(axis='both', which='major', labelsize=ticksize)
axs[1].set_xlim((10, 20))
axs[1].annotate('b)', xy=(0, 1), xycoords='axes fraction', xytext=(-50, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)

plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=figsize, gridspec_kw={'width_ratios': [1.6, 1]})

hist, xedge, yedge = ts.histogram2d_fast(df_dp_dt2['E_1'], df_dp_dt2['E_2'], range=((4, 11), (4, 11)), bins=70)
im = axs[0].imshow(hist, interpolation='none', origin='lower', extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]], aspect='auto')

# im = axs[0].hist2d(df_dt['E_1'], df_dt['E_2'], range=((4, 11), (4, 11)), bins=70)
axs[0].set_xlabel('E$_{1}$ (keV)', fontsize=labelsize)
axs[0].set_ylabel('E$_{2}$ (keV)', fontsize=labelsize)
axs[0].tick_params(axis='both', which='major', labelsize=ticksize)
axs[0].set_aspect('equal')  # Set aspect ratio to be equal for a square plot
cbar = fig.colorbar(im, ax=axs[0], shrink=.96)
cbar.ax.tick_params(labelsize=ticksize)
axs[0].annotate('a)', xy=(0, 1), xycoords='axes fraction', xytext=(-40, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)

axs[1].plot(df_dp_dt2_Etot_stats['h_centers'], df_dp_dt2_Etot_stats['h_counts'], '.', color='black')
axs[1].plot(df_dp_dt2_Etot_stats['model_x_vals'], df_dp_dt2_Etot_stats['model_y_vals'], '--', color='blue',
            label="$N_{{pairs}}$ = {:.0f} \u00B1 {:.0f} \n $E_0$ = {:.1f} keV \n  $\sigma$ = {:.2f}  keV".format(
                abs(df_dt_Etot_stats['N']), abs(df_dt_Etot_stats['N_err']), df_dt_Etot_stats['x0_2'], abs(df_dt_Etot_stats['sigma2'])))
axs[1].set_xlabel("E$_1$ + E$_2$ (keV)", fontsize=labelsize)
axs[1].set_ylabel("counts", fontsize=labelsize)
axs[1].tick_params(axis='both', which='major', labelsize=ticksize)
axs[1].set_xlim((10, 20))
axs[1].annotate('b)', xy=(0, 1), xycoords='axes fraction', xytext=(-50, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)

plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
fname = 'fig4b_'+plot_folder

fig, axs = plt.subplots(1, 2, figsize=figsize, gridspec_kw={'width_ratios': [1.6, 1]})

hist, xedge, yedge = ts.histogram2d_fast(np.degrees(df_dp_Etot2_dt['alpha_1']), np.degrees(df_dp_Etot2_dt['alpha_2']), range=((0.5, 1.6), (0.5, 1.6)), bins=70)

delta_theta_stats = ts.gauss_fit_series(np.degrees(df_dp_Etot2_dt['del_theta']), bin_size =0.0005, model_bin_size = 0.0001, srange=[0.015, 0.03])

im = axs[0].imshow(hist, interpolation='none', origin='lower', extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]], aspect='auto')

# im = axs[0].hist2d(df_dt['E_1'], df_dt['E_2'], range=((4, 11), (4, 11)), bins=70)
axs[0].set_xlabel("$alpha_{1}$ (deg)", fontsize=labelsize)
axs[0].set_ylabel("$alpha_{2}$ (deg)", fontsize=labelsize)
axs[0].tick_params(axis='both', which='major', labelsize=ticksize)
axs[0].set_aspect('equal')  # Set aspect ratio to be equal for a square plot
cbar = fig.colorbar(im, ax=axs[0], shrink=.96)
cbar.ax.tick_params(labelsize=ticksize)
axs[0].annotate('c)', xy=(0, 1), xycoords='axes fraction', xytext=(-40, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)
axs[0].set_xlim([0.5, 1.6])
axs[0].set_ylim([0.5, 1.6])



axs[1].plot(delta_theta_stats['h_centers'], delta_theta_stats['h_counts'], 'o', color='black', markersize = 2.0)
axs[1].plot(delta_theta_stats['model_x_vals'], delta_theta_stats['model_y_vals'], '--', color='grey', linewidth=1.0,
            label="$N_{{pairs}}$ = {:.0f} \u00B1 {:.0f} \n $\\Delta \\theta_0$ = {:.3f} deg \n  $\\sigma$ = {:.4f}  deg".format(
                abs(delta_theta_stats['N']), abs(delta_theta_stats['N_err']), delta_theta_stats['x0'], abs(delta_theta_stats['sigma'])))

axs[1].set_xlabel("Calculated detuning angle (deg)", fontsize=labelsize)
axs[1].set_ylabel("counts", fontsize=labelsize)
axs[1].tick_params(axis='both', which='major', labelsize=ticksize)
axs[1].set_xlim((0.015, 0.028))
axs[1].legend(loc='upper right', fontsize=4)
axs[1].annotate('d)', xy=(0, 1), xycoords='axes fraction', xytext=(-50, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)

plt.tight_layout()
# plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()

## detector acceptances

In [ ]:
fname = 'fig6_'+plot_folder

E1 = np.linspace(0.1,14.9,100)
x_cen = 258+2
y_cen = 257+2
cce, d1 = ts.conversion_efficiency(E1)
da = ts.detector_acceptance(d1, scan_info['x_cen'], scan_info['y_cen'])
tot = ts.tot_acceptance(E1)
ta = ts.total_acceptance(cce, da, tot)
datas = df_dt_Etot['E_1'].values
step = .5

counts0, edges0 = ts.histogram1d_fast(df_dp_Etot2_dt['E_1'], bins = np.arange(0, 15 + step, step), range=[0, 15])

bin_centers0 = (edges0[1:] + edges0[:-1]) / 2
counts1, edges1 = ts.histogram1d_fast(df_dp_Etot2_dt['E_2'], bins = np.arange(0, 15 + step, step), range=[0,15])
print(edges1)
counts_total = counts0 + counts1
bin_centers1 = (edges1[1:] + edges1[:-1]) /2 

errors0 = np.sqrt(counts0)
errors1 = np.sqrt(counts1)

plt.figure(figsize=figsize_sing)

plt.plot(E1, cce, '--', color='blue', label='Theory')
plt.errorbar(x=bin_centers0, y=counts_total / np.max(counts_total), yerr=np.sqrt(counts_total) / np.max(counts_total),
                ecolor='black', color='black', fmt='.', label='Exp. Data')
plt.plot(E1, ta[1], '--', color='darkorange',
            label='Model')
#plt.legend(fontsize=legendsize, loc='upper right')

plt.tick_params(axis='both', which='major', labelsize=ticksize)

plt.xlabel('single photon energy (keV)', fontsize=labelsize)
plt.ylabel('efficiency (normalized)', fontsize=labelsize)

if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

## ToT vs energy from radius

In [ ]:
fname = 'efig3a_'+plot_folder

data =np.load('/nsls2/data/chx/legacy/analysis/2023_2/qmicroscope/jhanrahan/chip_tot_info.npz')
modes1 = data['modes1']; modes2 = data['modes2']; modes3 = data['modes3']; modes4 = data['modes4']
means1 = data['means1']; means2 = data['means2']; means3 = data['means3']; means4 = data['means4']
energies = data['energy']

print(energies)

energies_f0 = energies[1:-1]
means4_f0 = means4[1:-1]

energies_f = np.concatenate(([2], energies[1:-1]))
means1_f = np.concatenate(([0], means1[1:-1]))
means2_f = np.concatenate(([0], means2[1:-1]))
means3_f = np.concatenate(([0], means3[1:-1]))
means4_f = np.concatenate(([0], means4[1:-1]))

# should move some of this to tpxspdc
def ftot(x, a, b, c, t):
    return a*x + b - c/(x-t)

def ftot_inv(y, a, b, c, t):
    numerator = a*t - b + y + np.sqrt(a**2 * t**2 + 2*a*b*t + 4*a*c - 2*a*t*y + b**2 - 2*b*y + y**2)
    return numerator / (2 * a)

p1opt, p1cov = curve_fit(ftot, energies_f, means1_f, p0 = [1/0.035, 100, 500, -1.5], maxfev = 1000000)
p2opt, p2cov = curve_fit(ftot, energies_f, means2_f, p0 = [1/0.035, 100, 500, -1.5], maxfev = 1000000)
p3opt, p3cov = curve_fit(ftot, energies_f, means3_f, p0 = [1/0.035, 100, 500, -1.5], maxfev = 1000000)
p4opt, p4cov = curve_fit(ftot, energies_f, means4_f, p0 = [1/0.035, 100, 500, -1.5], maxfev = 1000000)

e_dom = np.linspace(2, 17, 50)

plt.figure(figsize=figsize_sing)
plt.plot(energies_f[1:], means4_f[1:], '.', markersize=6, label='calibration', color='black')
plt.plot(e_dom, ftot(e_dom, *p4opt), '--', color='blue', label='a={:.2f}, b={:.1f}, c={:.1f}, t={:.2f}'.format(p4opt[0], p4opt[1], p4opt[2], p4opt[3]))
plt.plot([2], [0], marker='x', markersize='10', color='black')
plt.xlabel('Energy (keV)', fontsize=labelsize)
plt.ylabel('ToT (ns)', fontsize=labelsize)
plt.ylim([-20, 650])
plt.xlim([1.5,16.25])
plt.tick_params(axis='both', which='major', labelsize=ticksize)
plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
fname = 'efig3b_'+plot_folder

ch_no = 4

ch_1 = df_dp_Etot2_dt.loc[df_dp_Etot2_dt['chan_1'] == ch_no, ['ToT_sum_1', 'E_1']].values
ch_2 = df_dp_Etot2_dt.loc[df_dp_Etot2_dt['chan_2'] == ch_no, ['ToT_sum_2', 'E_2']].values
ch = np.concatenate((ch_1, ch_2))

plt.figure(figsize=figsize_sing)
# data = plt.hist2d(ch[:, 1], ch[:, 0], bins = [25,10], range=[[1,11],[150,400]])
data, xedge, yedge = ts.histogram2d_fast(ch[:, 1], ch[:, 0], bins = [25,10], range=[[1,11],[150,400]])
plt.imshow(data, origin='lower', extent=[xedge[0], xedge[-1], yedge[0], yedge[-1]],aspect='auto')
plt.plot(e_dom, ftot(e_dom, *p4opt), '--', color='red')
plt.xlim(1, 11)
plt.ylim(150, 400)
plt.xlabel('energy (keV)', fontsize=labelsize)
plt.ylabel('ToT (ns)', fontsize=labelsize)
plt.tick_params(axis='both', which='major', labelsize=ticksize)
plt.annotate('b)', xy=(0, 1), xycoords='axes fraction', xytext=(-40, 6), 
            textcoords='offset points', ha='left', va='top', fontsize=labelsize+4)
cbar = plt.colorbar(shrink=1)
cbar.ax.tick_params(labelsize=ticksize)
plt.tight_layout()
if save_plots: plt.savefig(plot_folder+'/'+fname+'.png', dpi=dpi)
plt.show()
plt.close()

In [ ]:
ts.entanglement_test(df_dp_Etot2_dt)

In [ ]:
ts.entanglement_test(df_dp_Etot2_dt, prints = False, plot = False)

In [ ]:
print("Set coordinate was [{}, {}]".format(set_x, set_y))
print("Best coordinate is most likely at {} with dp = {} pix and metric = {:.1f}".format(best_coord, best_dp, best_metric))
print("Optimal detuning angle found: {:.5f}".format(optimal_del_theta))
print("Time: N = {:.0f}, SN = {:.2f}".format(df_dt_stats['N'], df_dt_stats['SN']))
print("Time & Energy: N = {:.0f}, SN = {:.2f}".format(df_Etot2_dt_stats['N'], df_Etot2_dt_stats['SN']))
print("Time, Energy, Momentum: N = {:.0f}, SN = {:.1f}".format(df_dp_Etot2_dt_stats['N'], df_dp_Etot2_dt_stats['SN']))

In [ ]:
if first_run:
    log_string = f"sid={scan_info['sid']}:\nlen(_df)={len(_df)}, len(df)={df_length}, len(df_pairs)={df_pairs_length},\nfound_center=({scan_info['x_cen']},{scan_info['y_cen']}), opt_del_theta={optimal_del_theta:.5f},\nt_N={df_dt_stats['N']:.0f}, t_SN={df_dt_stats['SN']:.1f}, Et_N={df_Etot2_dt_stats['N']:.0f}, Et_SN={df_Etot2_dt_stats['SN']:.1f}, pEt_N={df_dp_Etot2_dt_stats['N']:.0f}, pEt_SN={df_dp_Etot2_dt_stats['SN']:.1f}\n\n"
    print(log_string)
    with open(log_file, 'a') as file:
        file.write(log_string)

In [ ]:
t15 = time.time()
print(f"Imports and setup done after {(t1-tstart):.1f} seconds.")
if first_run:
    print(f"Data loading complete after {(t2-t1):.1f} seconds.")
    print(f"Gap and additional centroid columns complete after {(t2b-t2):.1f} seconds.")
    print(f"Dead pixels removed after {(t3-t2b):.1f} seconds.")
    print(f"Data channeling complete after {(t4-t3):.1f} seconds.")
    print(f"Basic plots done after {(t5-t4b):.1f} seconds.")
    print(f"ToT filtering done {(t6-t5):.1f} seconds.")
print(f"Pairs found after {(t8-t7):.1f} seconds.")
print(f"Detuning info found after {(t9-t8):.2f} seconds.")
print(f"dp search concluded after {(t12-t11):.1f} seconds.")
print(f"Selections concluded after {(t14-t13):.1f} seconds.")
print(f"Plotting and logging complete after {(t15-t14):.1f} seconds.")
print(f"Total duration: {(t15-tstart):.1f} seconds.")

In [ ]:
import os
os._exit(0)